
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# 2.1 DEMO: Implementing Delta Sharing (Databricks-to-Databricks)

## Overview
In this demo, we will implement Delta Sharing to securely share data between Databricks workspaces. This demonstration is split into two parts:

**Provider Notebook:** We will create a Unity Catalog with sample customer and sales data, configure Delta Sharing by creating a share, set up a recipient organization, and generate an activation link for secure access.

**Recipient Notebook (This Notebook):** The receiving organization (using Databricks) will mount the share in a local catalog and query the shared tables directly without copying data, and perform analytics using the shared datasets.

This demo showcases the Databricks-to-Databricks (D2D) sharing pattern, where both the provider and recipient use Databricks workspaces. You'll see how Delta Sharing enables secure, live data sharing without data duplication or complex ETL processes.

## Background

**Scenario:**
You are a in the __West Division__ of a fictitious compant __*"Acme Corp"*__.  An administrator in the __East Division__ has shared their customer data with you. In this notebook, you'll set up the recipient side of Delta Sharing by mounting the Delta Share to a catalog to enable immediate access without copying or moving data.

In [0]:
%run ./Includes/Demo-Setup-2

## Step 1: View Available Providers

Execute the command below to show all of the providers you have access to on the metastore assigned to your workspace.  

⚠️ The recipient user mounting the share needs to have the `USE PROVIDER` privilege for the provider OR needs to be a __Metastore Admin__ for the metastore assigned to the recipient workspace.

In [0]:
-- show all providers available (that the calling user has USE PROVIDER privilege on)
SHOW PROVIDERS;

In [0]:
-- describe a provider
DESCRIBE PROVIDER acme_corp;

## Step 2: Show the Shares Available in the Provider

Execute the command below to show all of the shares available from the provider.  

In [0]:
-- show all of the shares available in a provider
SHOW SHARES IN PROVIDER acme_corp;

## Step 3: Mount the Share to a Catalog

On the **recipient side**, a privileged user must create a catalog from the share to make the shared data accessible. This catalog acts as a container for the shared tables and allows you to query them using standard SQL.

**Permissions required (recipient side):**
- Metastore admin, OR
- User with both `CREATE CATALOG` and `USE PROVIDER` privileges, OR  
- User with `CREATE CATALOG` privilege and ownership of the provider object

The catalog created from a share has a catalog type of "Delta Sharing" and provides read-only access to the shared tables. All shared data appears under the standard three-level namespace: `catalog.schema.table`.

**Note:** If the recipient workspace is using Databricks Community Edition (Free), these permissions are automatically granted.


In [0]:
-- mount the remote share to a local catalog (recipient side)
-- this creates a Delta Sharing catalog that provides read-only access to shared tables
CREATE CATALOG IF NOT EXISTS recipient_retail_catalog
USING SHARE acme_corp.internal_retail;

In [0]:
-- inspect catalog, note the Catalog Type is `Delta Sharing`
DESCRIBE CATALOG recipient_retail_catalog;

You will see your mounted share in __Delta Shares Received__ in the __Catalog__ view or in __Catalog Explorer__, the schema(s) - __retail__ in this case -  are inherited from the Delta Share.
<br />
<br />
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://github.com/stackql/databricks-data-sharing-and-collaboration/blob/main/images/recipient%20-%20delta%20shares%20received.png?raw=true"
    alt="Get Provider Name"
  >
</div>
<br />
<br />

## Step 4: Explore the Shared Data

Let's explore what's available in the shared catalog.

In [0]:
-- List all schemas in the shared catalog
SHOW SCHEMAS IN recipient_retail_catalog;

In [0]:
-- List all tables in the shared schema
SHOW TABLES IN recipient_retail_catalog.retail;

## Step 4: Query Shared Tables

Now let's query the shared data. Note that you have read-only access.

In [0]:
-- Now select from the customers table
SELECT * FROM recipient_retail_catalog.retail.customers;

In [0]:
-- THIS WONT WORK...
/* DELETE FROM recipient_retail_catalog.retail.customers; */

## Next Steps

You can now use the share directly for analytics or to join or union with data in another location (another catalog and schema), including creating views referencing the shared data.  We will cover replication strategies in the next module.

## Summary

✅ **What we accomplished:**

1. Created a catalog in our workspace from the share
2. Explored the shared schemas and tables
3. Queried shared data with full SQL capabilities
4. Understood recipient limitations (read-only access)

**Key Benefits:**
- 🚀 **No Data Copying**: Query data directly from the provider
- 🔒 **Secure**: Provider controls access and can revoke at any time
- ⚡ **Real-time**: Always access the latest data from the provider
- 💰 **Cost-effective**: No storage costs for recipients
- 🔧 **Standard SQL**: Use familiar SQL syntax for queries

In [0]:
-- drop the mounted share for workspace cleanup
DROP CATALOG recipient_retail_catalog;